In [0]:
%run /Repos/gowdhaman.bj@diggibyte.com/databricks_assignment/source_to_bronze/utils

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In gold notebook employee_silver_to_gold, call utils notebook in this notebook 
 Read the table stored in a silver layer as DataFrame and select the columns based on the following requirements. 

In [0]:
path = "databricks_assignment.employe_info.dim_employee"
file_format = "delta"

In [0]:
# Read Silver table
emp_tab_df = spark.read.table("databricks_assignment.employe_info.dim_employee")
display(emp_tab_df)

In [0]:
country_csv_file = '/Volumes/databricks_assignment/documents/csvfiles/Country-Q1.csv'
department_csv_file = '/Volumes/databricks_assignment/documents/csvfiles/Department-Q1.csv'
file_format = "csv"
# Read CSV files
country_df = read_file(country_csv_file,file_format,header=True,inferSchema=True)
department_df = read_file(department_csv_file,file_format,header=True,inferSchema=True)


In [0]:
display(emp_tab_df)
display(country_df)
display(department_df)

Find the salary of each department in descending order. 

In [0]:
df_join_salary = emp_tab_df.join(department_df, emp_tab_df["Department"] == department_df["DepartmentID"], "inner")

df_join_salary.groupBy("DepartmentName").agg(
    sum("Salary").alias("Total Salary")
).orderBy("Total Salary", ascending=False).display()

Find the number of employees in each department located in each country.

In [0]:
df_country_join = df_join_salary.join(country_df, emp_tab_df["Country"] == country_df["CountryCode"], "inner")

In [0]:
display(df_country_join)

In [0]:
df_country_join.groupBy("CountryName","DepartmentName").agg(count(col("employee_id")).alias("Employee_count")).sort("Employee_count").display()

List the department names along with their corresponding country names

In [0]:
df_country_join.select("CountryName","DepartmentName").display()

What is the average age of employees in each department? 

In [0]:
df_country_join.groupBy("DepartmentName").agg(avg("age").alias("average_employee_age")).sort("average_employee_age").display()

Add the at_load_date column to data frames.

In [0]:
def add_load_date(dataframe):
    return dataframe.withColumn("load_date",lit(current_date()))

employee_load_date = add_load_date(emp_tab_df)
country_df_load_date = add_load_date(country_df)
department_df_load_date = add_load_date(department_df)

In [0]:
display(employee_load_date)
display(country_df_load_date)
display(department_df_load_date)